<a href="https://colab.research.google.com/github/JuanM-GG/Biologia-de-sistemas/blob/main/Algoritmo_gen%C3%A9tico_completo_TSP.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Introduction
Genetic algorithms work by iterating over generations of populations and evaluation how well these populations solve a problem. At the end of a generation, the best individuals are selected to produce the next generation. 

The code presented in this blog has been adapted from:


**Zaccone G. (2019). Natural Computing with Python: Learn to implement genetic and evolutionary algorithms for problem solving in a pythonic way. BPB Publications.**



# Imports
First we import all necessary modules.

In [2]:
import numpy as np, random, operator, pandas as pd, matplotlib.pyplot as plt

# Data Representation
Because we are going to work with the simplest form of the TSP, we need a way to represent the data:

In [3]:
#Create class to handle "cities"
class City:
    # constructor
    def __init__(self,name, x, y):
        self.name = name
        self.x = x
        self.y = y
    
    # módulos 
    def distance(self, city):
        xDis = abs(self.x - city.x)
        yDis = abs(self.y - city.y)
        distance = np.sqrt((xDis ** 2) + (yDis ** 2))
        return distance
    
    def __repr__(self):
        return "(" + str(self.name)+ ")"


# How the algorithm works?

## The concept of individual
An individual can be seen as a single instance of the problem, for this case it's easy to see that the individual is the sequence of "cities" and the order that they are visited.

## Fitness
Genetic algorithms mimic natural structures using the idea of "Survival of the fittest", so it's important to define a common fitness function for all individuals. For this case, the fitness of an individual is the sum of the distance for each pair of consecutive cities, including the sum of the last city in the sequence and the first (because TSP forms a loop).
>$Fitness=[\sum_{i=1}^{N-1} Distance(City_{i},City_{i+1})]+Distance(City_{N},City_{1})$

This "Distance" function is just the euclidian distance between each city.

In [4]:
#Create a fitness function

class Fitness:
    # constructor
    def __init__(self, route):
        # atributos de la clase
        self.route = route
        self.distance = 0
        self.fitness= 0.0
    
    # módulos para calcular los atributos distance y fitness
    def routeDistance(self):
        if self.distance == 0:
            pathDistance = 0
            for i in range(0, len(self.route)):
                fromCity = self.route[i] # self.route[i] pertenece a la clase City y por lo tanto tiene el módulo distance
                toCity = None
                if i + 1 < len(self.route):
                    toCity = self.route[i + 1]
                else:
                    toCity = self.route[0]
                pathDistance += fromCity.distance(toCity) # calcula la distancia entre fromCity y toCity
            self.distance = pathDistance # suma esta distancia a las distancias anteriores
        return self.distance
    
    def routeFitness(self):
        if self.fitness == 0:
            self.fitness = 1 / float(self.routeDistance())
        return self.fitness

## Mutation and Breeding
The algorithm creates the next generations using two methods, either mutating single individuals based on a probability, or "mating" two individual to create a new one. Here we define all necesarry functions to perform the algorithm.

In [23]:
#puedo crear una lista de tuplas 
[(1,2), (3,4),(5,6)]

[(1, 2), (3, 4), (5, 6)]

In [26]:
#Create our initial population
#Route generator
#This method randomizes the order of the cities, this mean that this method creates a random individual.
def createRoute(cityList):
    route = random.sample(cityList, len(cityList))
    return route

In [ ]:
#Create first "population" (list of routes)
#This method creates a random population of the specified size.

def initialPopulation(popSize, cityList):
    population = []

    for i in range(0, popSize):
        population.append(createRoute(cityList))
    return population

In [33]:
population = initialPopulation(3, cityList)

In [46]:
#Rank individuals
#This function takes a population and orders it in descending order using the fitness of each individual
def rankRoutes(population):
    fitnessResults = {}
    for i in range(0,len(population)):
        fitnessResults[i] = Fitness(population[i]).routeFitness()
                        # contiene la ruta y su fitness        0->populaion, 1->fitness
    sorted_results=sorted(fitnessResults.items(), key = operator.itemgetter(1), reverse = True) # ordena de mayor a menor fitness
    return sorted_results

In [47]:
rankRoutes(population)  

[(1, 0.00042031301267919126),
 (2, 0.00040152545203373836),
 (0, 0.0003998360532906917)]

In [42]:
type(fitnessResults.items())

dict_items

In [49]:
df = pd.DataFrame(np.array(rankRoutes(population)), columns=["Index","Fitness"])
df

,Index,Fitness
0,1.0,0.000420
1,2.0,0.000402
2,0.0,0.000400


In [51]:
df['cum_sum'] = df.Fitness.cumsum()
df['cum_perc'] = 100*df.cum_sum/df.Fitness.sum()
df

,Index,Fitness,cum_sum,cum_perc
0,1.0,0.000420,0.000420,34.404664
1,2.0,0.000402,0.000822,67.271475
2,0.0,0.000400,0.001222,100.000000


In [ ]:
#Create a selection function that will be used to make the list of parent routes
 # popRanked -> index, fitness; # popRanked es un directorio
def selection(popRanked, eliteSize):
    selectionResults = []
    df = pd.DataFrame(np.array(popRanked), columns=["Index","Fitness"])
    df['cum_sum'] = df.Fitness.cumsum()
    df['cum_perc'] = 100*df.cum_sum/df.Fitness.sum()
    
    for i in range(0, eliteSize):
        selectionResults.append(popRanked[i][0]) # 0->index
    for i in range(0, len(popRanked) - eliteSize):
        pick = 100*random.random()
        for i in range(0, len(popRanked)):
            if pick <= df.iat[i,3]:
                selectionResults.append(popRanked[i][0])
                break
    return selectionResults

In [ ]:
#Create mating pool

def matingPool(population, selectionResults):
    matingpool = []
    for i in range(0, len(selectionResults)):
        index = selectionResults[i]
        matingpool.append(population[index])
    return matingpool

In [ ]:
#Create a crossover function for two parents to create one child
def breed(parent1, parent2):
    child = []
    childP1 = []
    childP2 = []
    
    geneA = int(random.random() * len(parent1))
    geneB = int(random.random() * len(parent1))
    
    startGene = min(geneA, geneB)
    endGene = max(geneA, geneB)

    for i in range(startGene, endGene):
        childP1.append(parent1[i])
        

    childP2 = [item for item in parent2 if item not in childP1]
    print(startGene, endGene)

    print(parent1)
    print(parent2)

    print(childP1)
    print(childP2)
    child = childP1 + childP2

    print(child)
    return child

In [ ]:
#Create function to run crossover over full mating pool
            # matingpool está ordenado del mejor al peor
def breedPopulation(matingpool, eliteSize):
    children = []
    length = len(matingpool) - eliteSize
    pool = random.sample(matingpool, len(matingpool)) # interesting function

    for i in range(0,eliteSize):
        children.append(matingpool[i])
    
    for i in range(0, length): # los de enmedio no se alcanzarán a seleccionar porque length < len(matingpool)
    # y se empieza con los extremos
        child = breed(pool[i], pool[len(matingpool)-i-1])
        children.append(child)
    return children

In [ ]:
#Create function to mutate a single route
def mutate(individual, mutationRate):
    for swapped in range(len(individual)):
        if(random.random() < mutationRate):
            swapWith = int(random.random() * len(individual))
            
            city1 = individual[swapped]
            city2 = individual[swapWith]
            
            individual[swapped] = city2
            individual[swapWith] = city1
    return individual

In [ ]:
#Create function to run mutation over entire population

def mutatePopulation(population, mutationRate):
    mutatedPop = []
    
    for ind in range(0, len(population)):
        mutatedInd = mutate(population[ind], mutationRate)
        mutatedPop.append(mutatedInd)
    return mutatedPop

In [25]:
#def mutatePopulation(population, mutationRate):
#    mutatedPop = []

#    for ind in population:
#        mutatedInd = mutate(individual, mutationRate)
#        mutatedPop.append(mutatedInd)
#    return mutatePop


#Put all steps together to create the next generation

def nextGeneration(currentGen, eliteSize, mutationRate):
    popRanked = rankRoutes(currentGen)
    selectionResults = selection(popRanked, eliteSize)
    matingpool = matingPool(currentGen, selectionResults)
    children = breedPopulation(matingpool, eliteSize)
    nextGeneration = mutatePopulation(children, mutationRate)
    return nextGeneration

In [6]:
x = {("a")}

In [7]:
df = pd.DataFrame([[0, 2, 3], [0, 4, 1], [10, 20, 30]],
                  columns=['A', 'B', 'C'])
# Access a single value for a row/column label pair.
df.iat[1,2]

1

In [8]:
child = []
childP1 = []
childP2 = []
parent1 = ["a","b","c","d"]
parent2 = ["b","c","d","a"]
geneA = int(random.random() * len(parent1))
geneB = int(random.random() * len(parent1))
startGene = min(geneA, geneB)
endGene = max(geneA, geneB)

for i in range(startGene, endGene):
    childP1.append(parent1[i])
    

childP2 = [item for item in parent2 if item not in childP1]
print(startGene, endGene)

print(parent1)
print(parent2)

print(childP1)
print(childP2)
child = childP1 + childP2

print(child)

0 0
['a', 'b', 'c', 'd']
['b', 'c', 'd', 'a']
[]
['b', 'c', 'd', 'a']
['b', 'c', 'd', 'a']


In [9]:
x = [0,1]
#random.sample(x, 6) it doesn't work 
list(np.random.randint(0,2,6)) # it works 

[0, 0, 0, 0, 0, 1]

## The genetic algorithm
With all these function defined, all that is left is to write the definition of the genetic algorithm.

In [10]:
#Final step: create the genetic algorithm

def geneticAlgorithm(population, popSize, eliteSize, mutationRate, generations):
    pop = initialPopulation(popSize, population)
    progress = [1 / rankRoutes(pop)[0][1]] # 1 entre fitness (distancia) del mejor actual
    print("Initial distance: " + str(progress[0]))
    
    for i in range(1, generations+1):
        
        pop = nextGeneration(pop, eliteSize, mutationRate)
        progress.append(1 / rankRoutes(pop)[0][1]) # 1 entre fitness (distancia) del mejor actual
        if i%50==0: # reportar cada 50 iteraciones i%n te da el númerador de la fracción; i//n te da el entero
          print('Generation '+str(i),"Distance: ",progress[i])
        
        
    bestRouteIndex = rankRoutes(pop)[0][0] # dónde está el más chingón 
    bestRoute = pop[bestRouteIndex] # su fitness
    
    plt.plot(progress) # progress tiene todas las mejores distancias de cada iteración
    plt.ylabel('Distance')
    plt.xlabel('Generation')
    plt.title('Best Fitness vs Generation')
    plt.tight_layout()
    plt.show()

    
    return bestRoute

In [11]:
5%2

1

## Testing
Know we create a list of cities and run the algorithm, this should return the best route found in the last generation.

In [29]:
#Running the genetic algorithm
#Create list of cities

cityList = []

for i in range(0,25):
    cityList.append(City(name = i, x=int(random.random() * 200), y=int(random.random() * 200)))

best_route=geneticAlgorithm(population=cityList, popSize=100, eliteSize=20, mutationRate=0.01, generations=500)
x=[]
y=[]
for i in best_route:
  x.append(i.x)
  y.append(i.y)
x.append(best_route[0].x)
y.append(best_route[0].y)
plt.plot(x, y, '--o')
plt.xlabel('X')
plt.ylabel('Y')
ax=plt.gca()
plt.title('Final Route Layout')
bbox_props = dict(boxstyle="circle,pad=0.3", fc='C0', ec="black", lw=0.5)
for i in range(1,len(cityList)+1):
  ax.text(cityList[i-1].x, cityList[i-1].y, str(i), ha="center", va="center",
            size=8,
            bbox=bbox_props)
plt.tight_layout()
plt.show()

In [21]:
cityList[1].y


167